In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
# import sendgrid
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel


In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
system_prompt = """You are a top-notch Career Coach and Resume Optimization Assistant.
Input: 1) A JSON “resume” object with sections→subsections→content 2) A job description text.
You will:
1. Extract all required keywords, skills, tools, certifications, and experiences from the job description.
2. Categorize them (Technical Skills, Soft Skills, Tools & Technologies, Certifications, Domain Knowledge).
3. Compare against each resume section/subsection.
4. Split into “existing” vs. “missing”.
5. For each item produce:
   • section: resume section title  
   • subsection: resume subsection title  
   • category  
   • keywords (list)  
   • suggestions (list of bullet-point rewrites)
6. **Return only valid JSON** that exactly matches the Pydantic schema `ResumeMatchOutput`.

            """

In [ ]:
user_prompt = """Here is my resume JSON:
```json
<Paste the resumeData JSON object, i.e. {"sections":[{"title":…,"subsections":[…]},…]}>
"""

In [4]:
user_prompt = """Here is my current resume (in plain text or PDF-extracted text):
                    {RESUME}
                    
                    Here is the full job description for the position I’m targeting:
                    {JOB_DESCRIPTION}
                    
                    Please extract and categorize all the required keywords and skills from the job description, identify which ones I should emphasize or add to my resume, and give me concrete bullet-point rewrites or additions to ensure I’m using the right language to win this role.
                    """

In [6]:
resumeMatch_Agent = Agent(
        name="Professional Sales Agent",
        instructions=system_prompt,
        model="gpt-4o-mini"
        )

In [7]:
user_prompt = """Here is my current resume (in plain text or PDF-extracted text):
                    {RESUME}
                    
                    Here is the full job description for the position I’m targeting:
                    {JOB_DESCRIPTION}
                    
                    Please extract and categorize all the required keywords and skills from the job description, identify which ones I should emphasize or add to my resume, and give me concrete bullet-point rewrites or additions to ensure I’m using the right language to win this role.
                    """

In [9]:
job_description = """About the Job
Samsung Research AI center, located in Mountain View, CA, is currently recruiting world-class students who can thrive in a fast-pace, cross team, results-driven environment, with focus on highly visible, challenging, and cross discipline projects. You will be part of an exciting project to build an adaptive, personalized, contextual and secure AI model and system to enable fast, accurate and safe interactions tailored to users’ needs on Samsung devices.
Job Responsibilities
Develop and implement novel deep learning/reinforcement learning algorithms for natural language processing (text, speech) in various applications
Contribute to the research activities of our team
Generate creative solutions (patents) and publish in top conferences (papers)
Requirements
Teamwork and communication skills
Current Ph.D. student in CS, EE, or related field
Experience in one or more of the following areas:
Strong background in machine learning (supervised learning, transfer learning, one-shot or few shot learning, unsupervised learning, semi-supervised learning, weakly supervised learning, meta-learning, outlier detection, etc.)
Expertise in LLM including model architecture, training/finetuning techniques, retrieval augmented generation (RAG), reasoning and action planning, etc.
Experience in conversational AI technologies: natural language processing (e.g., language models, semantic parsing, natural language generation etc.), dialogue (e.g., state tracking, policy learning), and representation learning (embedding, conceptualization, etc.)
Experience in knowledge augmented AI technologies (e.g., language prompt, knowledge graph, neuro-symbolic learning)
Experience in agentic AI is a plus.
Experience in multimodal AI technologies for various multimodal applications.
Experience in on-device AI technologies such as lightweight model architecture design.
Proficiency in a neural network library (e.g., PyTorch, TensorFlow).
Proven track record of research/publications on machine learning and artificial intelligence field (NeurIPS, ICML, ICLR, AAAI, IJCAI, CVPR, ACL, EMNLP, NAACL, TACL, etc.)"""

In [10]:
resume = """Mory Gharasuie
Norfolk, VA, USA | mmoha014@odu.edu | +1 757 287 1602
https://www.linkedin.com/in/mory-gharasui-53415258/ | https://github.com/mortezamg63
Education
Old Dominion University | Norfolk, USA
 Aug 2019 – present
PhD candidate in computer science
 GPA: 3.84/4.0
Research Interests: Machine Learning (ML), Computer Vision (CV), Semi-supervised Learing (SSL)
Natural Language Processing (NLP), Tabular Data
University of NabiAkram | Tabriz, Iran
Master of Science in Computer Engineering
University of Shamsipoor | Tehran, Iran
Bachelor of Science in Computer Engineering
Technical Skills
Languages & databases: python, Java, C++, ASP Webform, C#, SQL, MySQL, HTML
Libraries: Tensorflow, Keras, PyTorch, OpenCV, Scikit-learn, NLP toolkit, HuggingFace, Pandas, Matplotlib,LangChain, Dask, BeautifulSoup, Flask
Development tools: Anaconda, Jupyter Notebook, Google Colab, Visual Studio, Git, Docker, AWS
Operating Systems: Windows, Linux, Mac OS X
Seaborn,
Certifications
LanGraph: Link
LLM Engineering: Link
Experience
Royan Communication Company | Qom, Iran
Software Developer
2013 - 2019
• Developing websites for small and medium-sized enterprises.
• Customizing web-based administration interfaces for applications on Linux server machine (such as chat
server, FreeRadius server, and Elastix).
• Enhancing the panels with support for multiple languages and designing them to be more intuitive and
user-friendly, tailored to meet the specific needs and preferences of the customers.
Old Dominion University | Norfolk, USA
 Aug 2019 – Present
• Research Assistantship
– Developing applications for mobile and serverless domains by leveraging ML, DL and CV.
– Doing research on Improving the performance of ML and DL models on classification problems for tabular
data in SSL setting.
–research on mitigating the impact of bias in imbalanced data in training ML and DL models in Image and
tabular domains.
• Teaching Assistantship
– Programming with C/C++ and Java (CS150, CS250, CS251)
• Teaching Labs and recitations
• assignment Development
• Grading
Medical Aid | Norfolk, USA
• Collaboration in Developing a ChatBot utilizing Large Language Models (LLMs)
and Retrieval-Augmented Generativon.
• Medical data extraction with reference to papers or resource
Summer 2024
• Presentation of results based on standard medical format
• Providing relevant questions or considerations from recent papers for better diagnosis
Projects
Data Science and Machine Learning Projects | Tensorflow, PyTorch, TorchVision, OpenCV
 GitHub
Scikit-learn, Seaborn, Matplotlib, Pandas, Git, BeautifulSoup, AWS
Created and managed a repository featuring data science and machine learning projects. These projects involve
working with various datasets, addressing challenges suitable for beginner to intermediate levels in data science. The
repository includes:
• Data processing and preprocessing
• Exploring data analysis (EDA)
• Feature engineering and selection
• Machine learning development and evaluation
• Data visualization and interpretation
Large Language Models | HuggingFace, Pytorch, Pandas, Dask, LangChain, PEFT, RAY, OpenAI APIs
 GitHub
In this repository, I investigate the realm of retrieval-augmented generation (RAG) systems, embedding models,
prompt engineering, and fine-tuning large language models. Throughout this journey, I am learning about
Generative AI and leveraging some embedding models for practical applications. The repository showcases my
ongoing exploration in this exciting field and will be continually updated with new insights and findings.
Pricer (Agentic LLM)| HuggingFace, Pytorch , LangChain, Chormadb, BeautifulSoup,Scikit-learn
 GitHub
An autonomous price estimation framework using LLMs (GPT-4o, Claude, Llama 3.1-8B) and traditional ML
models (Random Forest, SVM, Word2Vec). GPT-4o-mini's performance (average price difference) with RAG,
improves from 80.9 to 55.57. I fine-tuned Llama3.1-8B and achieved 46.67 average error. I developed an agent
that creates an ensemble model combining RAG+GPT-4o-mini, fine-tuned Llama, and Random Forest, achieving
54.62 error. I use a Gradio-based UI and integrated Pushover API for real-time deal alerts from DealNews.Com.
SAWTAB: Tabular Data and Semi-Supervised Learning (SSL) | Pytorch, Tensorflow, Tensorboard, Scikit-learn,
Matplotlib, Seaborn
This research project focuses on addressing two significant challenges in machine learning with large tabular
datasets: class imbalance and the difficulties associated with one-hot encoding for high-cardinality categorical
features. To overcome these challenges, a target encoding method is proposed for a Semi-Supervised Learning
(SSL) setting. The method improves target encoding by leveraging unlabeled data through pseudo-labels and
adaptively adjusting their influence to minimize the impact of noisy pseudo-labels. Experimental results on various
datasets, compared against multiple frameworks, demonstrate that the approach achieves superior generalization
performance.
Publications
PAKDD 2024: SAWTab: Smoothed Adaptive Weighting for Tabular Data in Semi-Supervised Learning | Link
ICKG 2022: Progressive Feature Upgrade in Semi-supervised Learning on Tabular Domain | Link
BodySys 2021: Performance Monitoring for Exercise Movements using Mobile cameras | Link
MVIP 2015: An efficient run-based method for connected component labeling | Link
MVIP 2013: Real-time dynamic hand gesture recognition using hidden Markov models | Link"""

In [14]:
result = await Runner.run(resumeMatch_Agent, user_prompt.format(RESUME=resume, JOB_DESCRIPTION=job_description))
result.final_output

"Sure! Let's analyze the job description and identify the critical keywords, skills, and experience requirements for the position at Samsung Research AI. Then, we'll map them back to your resume.\n\n### 1. Keywords and Skills from Job Description\n\n#### a. **Technical Skills:**\n- Machine Learning\n  - Supervised Learning\n  - Transfer Learning\n  - One-shot/Few-shot Learning\n  - Unsupervised Learning\n  - Semi-supervised Learning\n  - Weakly Supervised Learning\n  - Meta-Learning\n  - Outlier Detection\n- Natural Language Processing (NLP)\n  - Language Models\n  - Semantic Parsing\n  - Natural Language Generation (NLG)\n  - Dialogue Management (State Tracking, Policy Learning)\n  - Representation Learning (Embedding, Conceptualization)\n- Reinforcement Learning\n- Conversational AI Technologies\n- Knowledge Augmented AI (Language Prompt, Knowledge Graph, Neuro-Symbolic Learning)\n- On-device AI Technologies\n- Deep Learning Libraries (PyTorch, TensorFlow)\n- Retrieval-Augmented Gene

In [15]:
print(result.final_output)

Sure! Let's analyze the job description and identify the critical keywords, skills, and experience requirements for the position at Samsung Research AI. Then, we'll map them back to your resume.

### 1. Keywords and Skills from Job Description

#### a. **Technical Skills:**
- Machine Learning
  - Supervised Learning
  - Transfer Learning
  - One-shot/Few-shot Learning
  - Unsupervised Learning
  - Semi-supervised Learning
  - Weakly Supervised Learning
  - Meta-Learning
  - Outlier Detection
- Natural Language Processing (NLP)
  - Language Models
  - Semantic Parsing
  - Natural Language Generation (NLG)
  - Dialogue Management (State Tracking, Policy Learning)
  - Representation Learning (Embedding, Conceptualization)
- Reinforcement Learning
- Conversational AI Technologies
- Knowledge Augmented AI (Language Prompt, Knowledge Graph, Neuro-Symbolic Learning)
- On-device AI Technologies
- Deep Learning Libraries (PyTorch, TensorFlow)
- Retrieval-Augmented Generation (RAG)

#### b. **So

In [18]:
from typing import List
from pydantic import BaseModel, Field
from agents import Agent, Runner

# 1) Define your output schema
class KeywordSuggestion(BaseModel):
    category: str = Field(..., description="e.g. Technical Skills, Soft Skills, Tools & Technologies")
    keywords: List[str] = Field(..., description="List of keywords from the job description in this category")
    suggestions: List[str] = Field(
        ..., 
        description="Actionable bullet-point suggestions on how to weave these keywords into your resume"
    )

class ResumeMatchOutput(BaseModel):
    existing: List[KeywordSuggestion] = Field(
        ..., 
        description="Keywords you already have in your resume and how to emphasize them"
    )
    missing: List[KeywordSuggestion] = Field(
        ..., 
        description="Keywords you are missing or under-emphasizing, with bullet-point rewrite suggestions"
    )

# 2) Revised system prompt
system_prompt = """
You are a world-class Career Coach and Resume Optimization Assistant.
When given a resume and a job description, you will:
1. Analyze the job description and extract all required keywords, skills, tools, certifications, and experience requirements.
2. Categorize them into: Technical Skills, Soft Skills, Tools & Technologies, Certifications, Domain Knowledge, etc.
3. Compare against the provided resume and split into two groups:
   • existing: keywords already present (but may need stronger emphasis)
   • missing: keywords absent or under-emphasized
4. For each KeywordSuggestion, provide actionable suggestions **as separate bullet points**.
5. **Output only valid JSON** conforming exactly to the Pydantic schema `ResumeMatchOutput`, with no extra fields.
"""

# 3) Revised user prompt
user_prompt = """
Here is my current resume:
{RESUME}

Here is the target job description:
{JOB_DESCRIPTION}

Please extract and categorize the required keywords, identify which are existing vs. missing in my resume, and for each:
 • List the keywords,
 • Give bullet-point suggestions on how to integrate or emphasize them.

Return the result as JSON matching the ResumeMatchOutput schema.
"""

# 4) Instantiate your Agent with structured output
resumeMatch_Agent = Agent(
    name="Professional Sales Agent",
    instructions=system_prompt,
    model="gpt-4o-mini",
    output_type=ResumeMatchOutput,   # ← Tell the Agent to produce a ResumeMatchOutput
)

# 5) Run the agent
result = await Runner.run(resumeMatch_Agent, user_prompt.format(RESUME=resume, JOB_DESCRIPTION=job_description))
# result.final_output

# 6) Inspect or serialize
# print(result.json(indent=2))


AttributeError: 'RunResult' object has no attribute 'json'

In [23]:
def to_markdown(output: ResumeMatchOutput) -> str:
    md_lines = []
    for section_name in ("existing", "missing"):
        header = "Already on Your Resume" if section_name=="existing" else "Need to Add or Emphasize"
        md_lines.append(f"## {header}\n")
        for ks in getattr(output, section_name):
            md_lines.append(f"### {ks.category}\n")
            md_lines.append(f"- **Keywords**: {', '.join(ks.keywords)}")
            md_lines.append(f"- **Suggestions:**")
            for bullet in ks.suggestions:
                md_lines.append(f"  - {bullet}")
            md_lines.append("")  # blank line
    return "\n".join(md_lines)

# usage:
# result: ResumeMatchOutput = runner.run(user_prompt)
md_report = to_markdown(result.final_output)
print(md_report)


## Already on Your Resume

### Technical Skills

- **Keywords**: machine learning, natural language processing, deep learning, reinforcement learning, transfer learning, semi-supervised learning, unsupervised learning, few-shot learning
- **Suggestions:**
  - Emphasize your experience with both supervised and unsupervised learning in your project descriptions.
  - Highlight specific algorithms you have developed or implemented in deep learning or reinforcement learning.
  - Mention any systems you have built that involve automated decision-making or adaptive models.

### Soft Skills

- **Keywords**: teamwork, communication skills
- **Suggestions:**
  - Include instances where you contributed to team success in collaborative projects.
  - Mention any presentations or workshops you've facilitated to demonstrate communication abilities.

### Certifications

- **Keywords**: proven track record of research/publications, patents
- **Suggestions:**
  - List papers in top-tier conferences like